The code in this colab notebook is heavily referenced from this [blog post by Usman Malik](https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras-part-2/).

# Imports and Setups

In [4]:
## save the installed packages in the google drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

#!pip install -qq wandb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Create a Weigts and Biases account if you don't already have one: https://wandb.ai/signup

Copy and paste the API token in the input box below: https://wandb.ai/authorize

In [6]:
import wandb
from wandb.keras import WandbMetricsLogger

## API key: 0f3669695b37815ed4c7636b6aece0b17875acb0
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# One to Many Sequence

## Data

In [ ]:
X, Y = [], []
X = [x+3 for x in range(-2, 43, 3)]

for i in X:
    output_vector = []
    output_vector.append(i+1)
    output_vector.append(i+2)
    Y.append(output_vector)

print(X)
print(Y)

Let's reshape the input sequence to be of the shape `[batch, timesteps, feature]`.

In [ ]:
X = np.array(X).reshape(15, 1, 1)
Y = np.array(Y)

print(f"Shape of X: {X.shape} and shape of Y: {Y.shape}")

## Model

In [ ]:
def get_model():
    inputs = layers.Input(shape=(1,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(2)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

## Train with W&B

In [ ]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

## Prediction

In [ ]:
test_input = np.array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input)
print(test_output)

# Many to One Sequence

## Data

In [7]:
X = np.array([x+1 for x in range(45)])
X = X.reshape(15,3,1)

Y = []
for x in X:
    Y.append(x.sum())
Y = np.array(Y)

print(X)
print(Y)

[[[ 1]
  [ 2]
  [ 3]]

 [[ 4]
  [ 5]
  [ 6]]

 [[ 7]
  [ 8]
  [ 9]]

 [[10]
  [11]
  [12]]

 [[13]
  [14]
  [15]]

 [[16]
  [17]
  [18]]

 [[19]
  [20]
  [21]]

 [[22]
  [23]
  [24]]

 [[25]
  [26]
  [27]]

 [[28]
  [29]
  [30]]

 [[31]
  [32]
  [33]]

 [[34]
  [35]
  [36]]

 [[37]
  [38]
  [39]]

 [[40]
  [41]
  [42]]

 [[43]
  [44]
  [45]]]
[  6  15  24  33  42  51  60  69  78  87  96 105 114 123 132]


## Model

In [8]:
def get_model():
    inputs = layers.Input(shape=(3,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(1)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


## Train

In [9]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

wandb: Currently logged in as: diagreen (recession_wranglers). Use `wandb login --relogin` to force relogin


Epoch 1/1000
1/1 [==============================] - 7s 7s/step - loss: 3932.4050 - val_loss: 14569.7061
Epoch 2/1000
1/1 [==============================] - 0s 39ms/step - loss: 3907.0085 - val_loss: 14483.2031
Epoch 3/1000
1/1 [==============================] - 0s 56ms/step - loss: 3881.4255 - val_loss: 14394.9561
Epoch 4/1000
1/1 [==============================] - 0s 37ms/step - loss: 3855.5212 - val_loss: 14304.2549
Epoch 5/1000
1/1 [==============================] - 0s 37ms/step - loss: 3829.1211 - val_loss: 14210.7080
Epoch 6/1000
1/1 [==============================] - 0s 40ms/step - loss: 3802.1328 - val_loss: 14113.8936
Epoch 7/1000
1/1 [==============================] - 0s 36ms/step - loss: 3774.4563 - val_loss: 14013.3467
Epoch 8/1000
1/1 [==============================] - 0s 37ms/step - loss: 3745.9832 - val_loss: 13908.7686
Epoch 9/1000
1/1 [==============================] - 0s 36ms/step - loss: 3716.7102 - val_loss: 13799.4580
Epoch 10/1000
1/1 [=============================

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,1998
batch/learning_rate,0.001
batch/loss,0.00503
epoch/epoch,999


## Prediction

In [10]:
test_input = np.array([50,51,52])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[153.40439]]


# Many to Many Sequence

## Data

In [ ]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

print(X)
print(Y)

## Model

In [ ]:
def get_model():
    inputs = layers.Input(shape=(3,1))
    encoder = layers.LSTM(100, activation="relu", return_sequences=False)(inputs)
    repeat = layers.RepeatVector(3)(encoder)
    decoder = layers.LSTM(100, activation='relu', return_sequences=True)(repeat)
    outputs = layers.TimeDistributed(layers.Dense(1))(decoder)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

## Train

In [ ]:
run = wandb.init(project="lstm-keras")

_ = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3, callbacks=[WandbMetricsLogger(log_freq=2)])

run.finish()

## Prediction

In [ ]:
test_input = np.array([300, 305, 310])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)